## Ownership mapping for LAND/CAMP

In [ ]:
import pandas as pd

In [ ]:
# load test parcel data and category map
parcels = pd.read_csv('LAND_CustomParcels_Final_2021_Tim.csv')
categories = pd.read_csv('OwnershipMap.csv')

In [ ]:
# add new columns to fill in
parcels['test_ownership'] = ''
parcels['test_categories'] = ''

In [ ]:
# show headers (only for show)
list(parcels)

In [ ]:
# show list of query strings for Religious Organizations (only for show)
for index, category in categories.iloc[29:30].iterrows():
    true_owner = category['true_owner[123]'].split('|')
    print(true_owner)

In [ ]:
# example copmlex querty as vector (only for show)
b = ((parcels['true_owner1'].str.contains('FLORIDA CITY') & \
    ~parcels['true_owner1'].str.contains('AGENCY',na=False) & \
    ~parcels['true_owner1'].str.contains('COMMUNITY',na=False) & \
    ~parcels['true_owner1'].str.contains('AGCY',na=False)) & \
    ~parcels['true_owner2'].str.contains('REDEVELOPMENT',na=False) & \
    ~parcels['true_owner2'].str.contains('REDV',na=False))

### Stand alone script for the whole process

In [ ]:
import pandas as pd

# load test parcel data and category map
parcels = pd.read_csv('LAND_CustomParcels_Final_2021_Tim.csv')
categories = pd.read_csv('OwnershipMap.csv')

# add columns for new categories
parcels['test_ownership'] = ''
parcels['test_categories'] = ''
        
def create_condition(owner):
    '''Create a conditional array to filter the parcels dataframe for an ownership
    category using a formatted string from the OwnsershipMap file.'''
    owner = owner.split('&&')
    
    if '!' not in owner[0]: # simple CONTAIN conditional on true_owner1
        #print ('simple','|'+owner[0]+'|')
        lb = parcels['true_owner1'].str.contains(owner[0])
        
    else: # AND NOT CONTAIN conditionals on true_owner1
        owner1s = owner[0].split('!')
        #print ('owner1','|'+owner1s[0]+'|')
        lb = parcels['true_owner1'].str.contains(owner1s[0])
        for owner1 in owner1s[1:]:
            #print ('owner1 not','|'+owner1+'|')
            lb = lb & ~parcels['true_owner1'].str.contains(owner1,na=False)
        lb = (lb)       
    
    if len(owner) > 1:    
        for owner2 in owner[1:]: # AND [NOT] CONTAIN conditionals on true_owner2
            if '!' not in owner2:
                #print ('owner2','|'+owner2+'|')
                lb = lb & parcels['true_owner2'].str.contains(owner2)
            else:
                owner2s = owner2.split('!')
                #print ('owner2','|'+owner2s[0]+'|')
                lb = lb & parcels['true_owner2'].str.contains(owner2s[0],na=False)
                for owner2 in owner2s[1:]:
                    #print ('owner2 not','|'+owner2+'|')
                    lb = lb & ~parcels['true_owner2'].str.contains(owner2,na=False)
        lb = (lb)
            
    return lb

# loop through all the categories in the OwnershipMap file : .iloc[29:30].
for index, category in categories.iterrows():
    true_owner = category['true_owner[123]'].split('|')
    print(category['Ownership'])

    # create the conditional pandas array to filter rows in the parcels
    b = create_condition(true_owner[0])
    for sub_owner in true_owner[1:]:
        b = b | create_condition(sub_owner)
        
    # set the ownership and categories in the parcels with the conditional array
    parcels.loc[b,'test_ownership'] = category['Ownership']
    parcels.loc[b,'test_categories'] = category['Owner Categories']
    
    # give some feedback
    print(
        'original',len(parcels[parcels['Ownership']==category['Ownership']]['Ownership']),
        'test',len(parcels[parcels['test_ownership']==category['Ownership']]['Ownership'])
    )

### Experiments to do row by row comparison

In [ ]:
# set ownership
ownership = 'Larkin'
category = categories[categories['Ownership']==ownership]
true_owner = category['true_owner[123]'].values[0].split('|')

# create the conditional pandas array to filter rows in the parcels
b = create_condition(true_owner[0])
for sub_owner in true_owner[1:]:
    b = b | create_condition(sub_owner)

In [ ]:
parcels[b][['Ownership','true_owner1','true_owner2']]

In [ ]:
sub = parcels[b][['ogc_fid','Folio','Ownership','test_ownership','true_owner1','true_owner2','dor_code_cur']]

In [ ]:
len(sub)

In [ ]:
sub[sub['Ownership']!='Religious Organization']

In [ ]:
org = parcels[parcels['Ownership']=='Religious Organization']

In [ ]:
for index, line in org.iterrows():
    if line['ogc_fid'] not in sub['ogc_fid'].values:
        print (line['dor_code_cur'],line['ogc_fid'],line['true_owner1'],line['true_owner2'])

In [ ]:
parcels.columns